# Sequence to Sequence Model (with onehot encoder)
Hoyai 를 통해서 Seq2Seq 모델을 구성하는 방법을 설명하고자 한다. Hoyai 에서 제공하는 Seq2Seq 는 단어의 Embedding 방식을 Google 에서 제공하는 Default Embedding 을 사용하지 않고, 별도의 Word2Vec 를 구성하여 연동할 수 있도록 한다. 또한, 한글 POS 처리 등 부분들도 내부적으로 지원하여 한글에 대한 최적처리를 지원하고 있다. <br>
그럼 두가지 중요한 개념인 Seq2Seq 와 Word2Vec 에 대해서 간단하게 설명하도록 하겠다. 아래의 그림은 Seq2Seq 를 간단하게 설명하는 그림이다.그렇다면 Seq2Seq 는 어떤 경우에 활용할 수 있는 모델일까?<br><br>
<b>[번역]</b> 가장 대표적인 사용예는 번역이 될 것이다. 요즘 Google 번역기가 매우 좋아진 것도 바로 이런 이유이다. <br>
<b>Encode : 안녕하세요. 오늘 기분은 어떠세요?</b> <br>
<b>Decode : Hello. How are you feel today?</b> <br>
위와 같이 데이터를 구성하고 아래의 Network 을 이용하여 훈련을 시키면 "안녕하세요. 오늘 기분은 어떠세요?" 라고 입력을 하였을때, "Hello. How are you feel today?"라는 답을 하는 네트워크가 구성되는 것이다. <br><br>
<b>[대화]</b>번역뿐만 아니라 간단한 문/답에도 아래와 같이 적용해 볼 수 있을 것이다. (※단순 Seq2Seq 로 Alex 같은 것은 구현할 수 없다)<br>
<b>Encode : 안녕하세요!? </b> <br>
<b>Decode : 잘가세요!?</b> <br><br>
<b>[분류]</b>Time Series 한 Classification 문제도 생각해 볼 수가 있다. 아래의 데이터를 주가의 흐름이라고 하자 <br>
<b>Encode : 1900, 1800, 1700, 1600, 2000 </b> <br>
<b>Decode : Up/Down</b> <br>
예를 들면 종합 주가지수가 1900, 1800, 1700, 1600, 2000  와 같이 변동해 왔을때, 내일의 주가는 내려갈까요? 올라갈까요? 와 같은 형태의 질문도 훈련을 할 수가 있을 것이다. 

<br><br><b>[그림1] Seq2Seq <b> <br>
<img src="../../images/seq2seq_desc.png">

# NLP and Preprocessing 
데이터에 대한 전처리는 어떤 종류의 Deep Learning 을 시도하여도 매우 중요한 문제이겠지만, NLP 에서는 더욱더 Critical 하고 어렵고 복잡한 문제라고 개인적으로 생각한다. 데이터에 대한 전처리는 구문분석, Entity 분석, Word2Vec, POS Tag, Doc2Vec 등.. 많은 것들이 있을 수 있지만 우리는 Seq2Seq 를 효과적으로 구성하기 위하여 POS Tagging 과 Word2Vec 두 가지를 사용하였다. 기본적으로 문장을 신경망에 훈련할때는 아래와 같은 과정을 거치게 된다. <br><br>
<b>(1) Tockenizer:</b> 문장을 띄어쓰기, 마침표 등 기준으로 단어 단위로 분리를 해줘야 한다. <br>
<b>(2) POS Tag: </b>모든 언어가 그렇지만, 같은 단어가 상황에 따라 다르게 사용될 수 있기 때문에, Word/품사와 같이 분류해준다 <br>
<b>(3) Special Char :</b> 알수 없는 문자, Decoder부 시작, Padding 부분 등을 표시하는 Vecotr를 정의하고 앞뒤에 추가해 준다. <br>
<b>(4) 사전구성 :</b> (3)의 특수문자를 포함하여 사용할 단어와 Index 를 맵핑하는 형태로 사전을 전부 구성해야 한다. <br>
<b>(5) Embedding :</b> 신경망은 결론적으로 문자 자체를 입력으로 받을 수는 없다. 때문에 Vector 로 바꿔줘야한다.<br>
<b>※ 이때 사용할 수 있는 것이 OneHot Encoding 과 Word2Vecotr 이다. (두 가지의 차이가 매우 중요하다!) <b><br>

<br><br><b>[그림2] OneHot Encoder </b> <br>
<img src="../../images/onehot_encoder.png"> <br>
OneHot Encoder는 위에 그림에서 보는 것 처럼 사전에 포함된 단어의 수 만큼 Vector를 구성하고 해당 단어의 Index 에 대응되는 값만 1로 표시해주는 형태라고 보면 된다. 이런 방식의 문제는 (1) 단어의 수가 많아지면 Vector의 크기가 계속 커진다. (2) 단어간의 유사성을 표현할 수 없어 유사한 표현을 처리하기는 어렵다. 두가지 정도일탠데, (2) 번 사항은 실제로 Seq2Seq 모델을 만들었을때 굉장히 큰 성능의 차이를 보인다고 생각한다. <br>

※ 다만, 지금까지의 테스트 결과 W2V Model 이 잘 훈련되어 있지 않아, 단어간의 Vector 차이가 너무 적은 경우, 또 훈련하고자 하는 대화의 양은 적은데 Word2Vector 모델에서 관리하는 단어는 너무 많은 경우 오히려 Onehot 보다 성능이 나쁜 경우도 확인하였다. 하지만, 장기적으로 더 많은 대화를 이해하고 조금 다른 문맥도 이해하게 만들고자 한다면, 더 많은 데이터를 확보하고 Word2Vec 을 활용하는 것이 좋다고 생각한다. 

# Hoyai 에서 실행하기
실제로 데이터 전처리부터 Seq2Seq 모델까지 직접만드려고 하면 굉장히 많은 양의 코드를 개발하고 테스트 해야할 것이다. 하지만 Hoyai 에서는 굉장히 쉽게 API 를 통해서 모델을 만들 수 있도록 지원하고 있다. 그러면 필요한 Step 을 먼저 정리해 보자. 

<b>(1) Word2Vec 모델 :</b> 첫번째로 해야할 일은 Word2Vec 모델을 만드는 일이다. 상세한 방법은 <b>(상세설명 : [link to source])</b> 를 참조하면 된다.  <br> ※ Hoyai 에서는 각각의 모델이 별도로 관리되고 모델을 다른 모델에서 다시 활용 할 수 있도록 시스템이 구성되어 있다.<br>
<b>(2) Network 등록 :</b> 다시 Seq2Seq 로 돌아와서 비지니스 측면에서 모델을 생성한다. (상세설명 : [link to source](../01.common/common_neural_network_define.ipynb))</b> <br>
<b>(3) Network 버전 등록 :</b> 네트워크의 버전 및 Stage 를 등록한다.  <br>
<b>(4) Graph Flow 등록 :</b> 실행 내용을 정의한다.  <br>
<b>(5) Graph Node Parm 등록 :</b> 각 Node 에서 실행할 내용을 정의한다 <br>
<b>(6) Run Train :</b> 정의된 Graph 내용을 실행한다. <br>
<b>(7) Predict Service :</b> API 를 통해 예측 서비스를 제공할 수 있다. <br>

# (2)(3) Network 등록/버전 등록
해당 부분은 모든 네트워크가 동일하게 적용되는 부분으로 자세한 설명은 (상세설명 : [link to source](../01.common/common_neural_network_define.ipynb))를 참조 할 수 있도록 한다. 

In [4]:
import requests
import json, os

nn_id = 'nn000995'  # put some key value you want to test

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
print("done")

done


In [55]:
####(1) 네트워크 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

####(2) 버전 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : nn000995
evaluation result : nn000995


# (4) Graph 생성
실제로 어떤 순서로 어떻게 동작을 할 것인가를 구성하는 단계로 Graph 구성에 대한 추가적인 설명이 필요할 것이라고 생각된다. <br>
Network - Version - Stage 형태로 데이터가 구성되며, Stage 안에는 아래의 그림과 같이 개발 기능단위 컴포넌트 (앞으로는 Node 라고 지칭)를 정의하고 Relation 을 정의할 수 있다. Graph Flow 를 정의하고, 각 Node 의 동작 내용을 상세 정의하는 과정을 수행하면, 실제로 전체 Flow 를 실행할 준비가 끝나는 것이다. 


<br><img src="../../images/graph_flow.jpg"> <br>


# (4) Graph Flow 생성 
실제로 수행하고자하는 Flow 를 구성하기 위한 API 를 호출한다. Dynamic 하게 Flow 를 구성할 수도 있지만, 정해진 Flow 를 강제로 생성하기 위한 API 를 사용하도록 한다. 강제로 생성되는 Flow 는 아래와 같이 총 6개의 Node 로 구성되어 있으며, 각 Node 는 정의되어야 한다.  <br> 
<b>Train Data >> Feed >> Network << Evaluation << Test Data << Feed <b>
 

In [56]:
# Work Flow 틀을 구성하도로고 지시한다. (정해진 틀을 강제로 생성)
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/' + nn_id +'/wfver/1/',
                     json={
                         "type": 'seq2seq_csv'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : seq2seq_csv


# (5) 데이터 업로드 
데이터를 Jyupter를 통해서 업로드 하는 과정은 아래와 같다. <br>
1. <b>Jupyter Upload Button 을 활용하여 data 폴더에 데이터를 옮긴다.</b> <br>
※ 정확한 위치는 여기 :  /home/dev/hoyai_jupyter/data/ <br>
2. <b>Data Upload API 를 실행하여 실제 Src 폴더로 데이터를 이동시킨다.</b> <br>
※ 만약 자신의 Local 에서 Jupyter 를 실행하였다면, 2번만 바로 수행해도 될 것이다. <br> 
3. <b>테스트에서 사용하고자 하는 데이터 </b> <br> 
 (데이터 보기 : [link to source](../../data/seq2seq.csv))

In [57]:
import os

return_dict = {}
return_dict['test'] = open('../../data/seq2seq_friday.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_csv_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']


# (5) Node 속성 정의 - Train Data Node
Train 을 위한 데이터를 수집하고 전처리하여 저장하는 과정을 Node 로 정의한다. DataNode는 세부적으로 3가지 Step 으로 구성되어 있다. <br>
Frame Data Node 를 사용하는 경우 실질적인 POS Tagging 처리 등은 Feeder 에서 이루어 지게 된다. <br>
1. <b>데이터 수집에 대한 정의</b>  <br>
2. <b>데이터 전처리에 대한 정의</b> <br>
3. <b>데이터 저장에 대한 정의</b> <br>


In [58]:
# (1) Train Data Node 의 속성을 정의
# 어디서 Source 를 어떻게 가지고 올것인지 파라메터 정의 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_csv_node/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })

# 전처리는 어떤 것을 할지 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/data_csv_node/',
                      json={
                          "preprocess":  "none",
                      })
# 전처리가 완료된 데이터는 어디에 저장을 할지 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/data_csv_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/nn000995/1/data_csv_node


# (6) Node 속성 정의 - Data Feeder
Feeder 는 Network 와 Data Node 사이에서 Network 에서 사용하기 적합한 형태로 각종 가공 처리를 하기 위한 용도로 개발되었으며, 아래의 Node 의 경우 Frame Data 에서 Seq2Seq 모델로 데이터를 연결하기 위한 용도로 사용된다. <br>
본 예제에서는 csv 데이터를 읽어서 Seq2seq 에 Feed 해주는 형태로 어떤 csv 컬럼을 각각 Encode 부와 Decode 부로 사용할 것인지 정의하고 pos Tagging에 어떤 알고리즘을 사용할 것인지, 그리고 문장의 길이는 최대 어느정도까지 인지할 것인지 지정하도록 한다. <br>
- encode_column : Csv 파일에서 Encode 에 사용할 컬럼 명 <br>
- decode_column : Csv 파일에서 Decode 에 사용할 컬럼 명 <br>
- max_sentence_len : 문장의 길이를 최대 어디까지 인지할 것인지 지정 <br>
- preprocess : 사용할 Pos Tagger 를 지정 (mecab, kkma, twiter 등) <br>

In [59]:
# (2) Network 에 데이터를 Feed하는 Node 의 속성을 정의 
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/seq2seq/nnid/'+nn_id+'/ver/1/node/feed_fr2seq/',
                     json={
                         "encode_column" : "encode",
                         "decode_column" : "decode",
                         "encode_len" : 15,
                         "decode_len" : 3,
                         "preprocess": "mecab",
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'encode_len': 15, 'preprocess': 'mecab', 'decode_len': 3, 'encode_column': 'encode', 'decode_column': 'decode'}


# (7) Node 속성 정의 - Seq2Seq 신경망 정의
Seq2Seq 모델은 Encoder부와 Decoder 부로 정의되어 있으며, 아래와 같은 파라메터들을 정의할 수 있다. <br>
- encoder_len : 인코더 부분의 길이 지정 <br>
- decoder_len : 디코더 부분의 길이 지정 <br>
- encoder_depth : 인코더 Hidden Layer 깊이 <br>
- decoder_depth : 디코더 Hidden Layer 깊이 <br>
- cell_type : vanila, lstm, gru <br>
- cell_size : cell 의 Vector size <br>
- drop_out : Train  시 Drop Out Rate <br>
- word_embed_type : onehot encoder 를 사용할 것인지, word2vector를 사용할 것인지 판단 <br>
- word_embed_id : Word2Vector인 경우 사용한 기 훈련된 Network ID 를 지정해야 함  <br>
- batch_size : 한번에 훈련할 데이터 건수  <br>
- iter : 반복해서 훈련할 횟수  <br>
- early_stop : 지정횟수 이전에 훈련을 종료하기 위한 적중률 기준  <br>
- learning_rate : Weight 값 갱신시 사용한 Hyper Parameter  <br>

In [1]:
# update source_info
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/seq2seq/nnid/'+nn_id+'/ver/1/node/netconf_node/',
                     json={
                         "encoder_len" : 15,
                         "decoder_len" : 3,
                         "encoder_depth" : 2,
                         "decoder_depth" : 2,
                         "cell_type" : "lstm",   #vanila, lstm, gru
                         "cell_size" : 500,
                         "drop_out" : 0.8,
                         "word_embed_type" : "onehot",   #w2v, onehot
                         "word_embed_id" : "",
                         "vocab_size" : 200,
                         "batch_size" : 74,
                         "iter" : 100,
                         "early_stop" : 0.9,
                         "learning_rate" : 0.001
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

NameError: name 'requests' is not defined

# (8) Node 속성 정의 - Test Data Node
Test 를 위한 데이터와 Feeder 를 정의합니다. 세부 내용은 Train 을 위한 데이터 정의와 동일합니다. <br>

In [61]:
# upload files for test
return_dict = {}
return_dict['test'] = open('../../data/seq2seq_friday.csv', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/evaldata/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 소스 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/evaldata/',
                     json={
                         "type": "csv",
                         "source_server": "local",
                         "source_sql": "all",
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 전처리 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/evaldata/',
                      json={
                          "preprocess":  "none",
                      })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 저장 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/evaldata/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


# Feeder 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/frame/net/seq2seq/nnid/'+nn_id+'/ver/1/node/feed_fr2seq_test/',
                     json={
                         "encode_column" : "encode",
                         "decode_column" : "decode",
                         "encode_len" : 15,
                         "decode_len" : 3,
                         "preprocess": "mecab",
                     })
data = json.loads(resp.json())

node_name = 'eval_node'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/eval/nnid/'+nn_id+'/ver/1/node/'+node_name+'/',
                    json={
                        "type": "seq2seq",
                    })

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']
evaluation result : {'max_sentence_len': 0, 'source_path': '/hoya_src_root/nn000995/1/evaldata', 'source_type': 'local', 'type': 'csv', 'multi_node_flag': None, 'source_parse_type': 'raw', 'source_server': 'local', 'source_sql': 'all'}
evaluation result : none
evaluation result : /hoya_str_root/nn000995/1/evaldata
evaluation result : {'type': 'seq2seq'}


# (9) Train 수행
이제 훈련을 위한 준비는 모두 끝났다. 아래의 API 를 호출하면 지금까지 정의한 Graph 가 수행되며, 이 Graph 는 정주기 혹은 Event 에 의해서 자동적으로 수행되어 발생하는 데이터를 지속적으로 모델에 반영시킬 수 있도록 동작하게 된다. 

In [62]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/1/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'result': 'sequence item 0: expected str instance, list found', 'status': '404'}


# (10) Predict 수행
아까 훈련했던 데이터로 다시 테스트를 해보자... <br>
테스트 결과는 별다른 필터링 없이 Start, Unknown, Pad 등 도 모두 출력 되도록 일부러 필터링 로직을 바이패스 하였다. <br>
<table>
<tr><td>encode</td><td>decode</td></tr>
<tr><td>[이름]을 찾아줘! [이름][직급] 찾아주라 (정보검색) </td><td>2</td></tr>
<tr><td>[업무]담당자가 누구지 (담당자 검색)</td><td>3 </td></tr>
<tr><td>[이름]의 [날짜] [근태코드] 어디있지.</td><td>4</td></tr>
<tr><td>[이름]의 [직급]을 찾아줘</td><td>6</td></tr></tr>
<tr><td>[회의] 를 [이름] [이름] 참석자로 [시작시간] [끝시간] [장소]로 예약해줘</td><td>7</td></tr>
</table>

In [ ]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름]을 찾아줘" , "num": 3, "clean_ans":False}
                     )
data = json.loads(resp.json())
print("evaluation result(2) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름][직급]을 찾아주라" , "num": 3, "clean_ans":False}
                     )
data = json.loads(resp.json())
print("evaluation result(2) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "담당자 검색 해 줄레" , "num": 3, "clean_ans":False}
                     )
data = json.loads(resp.json())
print("evaluation result(3) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] 의 [직급] 찾 아 줘 " , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(6) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] [근태코드] 갔 어" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(4) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[이름] 의 [직급] 알 고 있 냐" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(6) : {0}".format(data))

resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/seq2seq/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "[회의] 를 [이름] [이름] 참석자로 예약해줘" , "num": 0, "clean_ans":True}
                     )
data = json.loads(resp.json())
print("evaluation result(7) : {0}".format(data)) 



evaluation result(2) : [['@', '2/SN', 'SF'], ['-1', '-1', '-1'], ['-1', '-1', '-1'], ['-1', '-1', '-1']]
evaluation result(2) : [['@', '2/SN', 'SF'], ['-1', '-1', '-1'], ['-1', '-1', '-1'], ['-1', '-1', '-1']]
evaluation result(3) : [['@', '3/SN', 'SF'], ['-1', '-1', '-1'], ['-1', '-1', '-1'], ['-1', '-1', '-1']]
